In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np

import qubex.pulse as qp
from qubex.simulator import Coupling, QuantumSimulator, QuantumSystem, Transmon

In [ ]:
transmons = [
    Transmon(
        label="Q0",
        dimension=2,
        frequency=7.0,
        anharmonicity=-0.333,
        # relaxation_rate=0.00005,
        # dephasing_rate=0.00005,
    ),
    Transmon(
        label="Q1",
        dimension=2,
        frequency=8.0,
        anharmonicity=-0.333,
        # relaxation_rate=0.00005,
        # dephasing_rate=0.00005,
    ),
]

couplings = [
    Coupling(pair=("Q0", "Q1"), strength=0.01),
]

system = QuantumSystem(
    objects=transmons,
    couplings=couplings,
)

In [ ]:
simulator = QuantumSimulator(system)

In [ ]:
control_qubit = transmons[0]
target_qubit = transmons[1]

In [ ]:
qp.set_sampling_period(0.1)

pi_pulse = qp.Rect(duration=20, amplitude=2 * np.pi * 0.025)
cr_pulse = qp.FlatTop(duration=200, amplitude=2 * np.pi * 0.2, tau=30)

with qp.PulseSchedule(
    targets=[
        "Q0",
        "Q0-Q1",
        "Q1",
    ],
    frequencies={
        "Q0": transmons[0].frequency,
        "Q0-Q1": transmons[1].frequency,
        "Q1": transmons[1].frequency,
    },
    objects={
        "Q0": transmons[0].label,
        "Q0-Q1": transmons[0].label,
        "Q1": transmons[1].label,
    },
) as ps:
    ps.add("Q0-Q1", cr_pulse)
    ps.barrier()
    ps.add("Q0", pi_pulse)
    ps.barrier()
    ps.add("Q0-Q1", cr_pulse.scaled(-1))
    ps.barrier()
    ps.add("Q0", pi_pulse)

ps.plot()

In [ ]:
result = simulator.mesolve(
    controls=ps,
    initial_state=system.state(
        {
            "Q0": "0",
            "Q1": "0",
        },
    ),
)

result.plot_bloch_vectors("Q0")
result.plot_bloch_vectors("Q1")

result.display_bloch_sphere("Q0")
result.display_bloch_sphere("Q1")

result.plot_population_dynamics("Q0")
result.plot_population_dynamics("Q1")
result.plot_population_dynamics()

In [ ]:
result = simulator.mesolve(
    controls=ps,
    initial_state=system.state(
        {
            "Q0": "1",
            "Q1": "0",
        },
    ),
)

result.plot_bloch_vectors("Q0")
result.plot_bloch_vectors("Q1")

result.display_bloch_sphere("Q0")
result.display_bloch_sphere("Q1")

result.plot_population_dynamics("Q0")
result.plot_population_dynamics("Q1")
result.plot_population_dynamics()